# Analyzing representative cycles for persitent homology with Giotto-TDA

**Authors:** Guoxi Liu, Federico Iuricich 

**Affiliation:** Clemson University

The goal of this notebook is to study the interpretation of the persistent homology with the cycle information.

We investigate the cycle information of the persistent homology on a subset of MNIST dastaset, and introduce possible applications from the direct visualization of the cycle information.


# 1. Introduction and Motivation

Information captured by persistent homology is commonly represented by a persistence diagram or, equivalently, a persistence barcode. 

These are elegant mathematical representations of the homology classes that appear and disappear in a filtration. However, their use for exploratory data analysis is challenging. While they effectively represent the lifespan of each homology class, they also lose connection with the original domain of data.  

This brings to two main problems:

1. we have no information regarding the distribution of persistence pairs in the original domain
2. we have no information regarding the shape of each homology class appearing in the filtration.

This means that interpreting the results becomes extremely difficult for a user. Experts in TDA may be able to guess this information by combaning persistence diagrams with a visualization of the original filtration. However, this is generally not an easy task, especially for newcomers. 

Questions like

- Is the filtration I am using correct?
- Are the homology classes appearing in my persistence diagram aligned with the features I see in the input data?

are quite natural when we start computing persistent homology on a new dataset. 


To be able to answer this type of questions we need to map information from the persistence diagram back to the original domain of the filtration. In this notebook, we provide a collection of functions to simplify the interactive visualization and analysis of homology class by enriching the information contained in the persistence diagram with cycles.

# 2. Analysis

## 2.1. Dataset description

![dataset](./pictures/dataset.png)

In the remainder of this notebook, we use a sample of the original MNIST dataset downloaded from [here](http://yann.lecun.com/exdb/mnist/). The dataset provided in this submission includes 500 handwritten digits (50 images per digit) that have been size-normalized and centered in a fixed-size image.


## 2.2. Preprocessing

The first step when using persistent homology is that of defining a filtration on the input data. This already provides a degree of freedom for users that may define the most suitable filtration based on the features they want to be highlighted.

For example, in the case of an MNIST digit we may want to highlight dominant cycles in the input digits. To this end, we may use different filtrations:
- the one defined by the original grayscale image, 
- the inverse(negated) grayscale image,
- or new image obtained by applying the distance transform to the input image

to name a few. Clearly, each image will be transformed in a filtration generating different homology cycles and, consequently, different persistence diagrams. In this notebook, all filtrations are generated applying the distance transform to the input image, but any other filtration could be used instead.

![filtrations](pictures/filtrations.png)
Images generated with `./scripts/filtrations.py`

### External tools

The computation of persistence diagrams and representative cycles has been implemented in a plain C++ [library](https://github.com/IuricichF/PersistenceCycles) integrated in the Topology Toolkit (TTK) [TTK](https://topology-tool-kit.github.io). For simplicity, we have precomputed all persitence diagrams and all representative cycles with our external library and saved them in the folder `data/vtk-*/`.

## 2.3. Visualizying cycles

We import necessary Python packages.

In [1]:
import sys

# Check Python version and make sure the version >= 3.8.0 
print(sys.version)

# Install dependencies 
!{sys.executable} -m pip install meshio
!{sys.executable} -m pip install giotto-tda
!{sys.executable} -m pip install scikit-image
!{sys.executable} -m pip install pandas

3.8.0 (default, Feb 25 2021, 22:10:10) 
[GCC 8.4.0]
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


The functions provided in this notebook allow for the easy visualization and comparison of homology classes by means of the representative cycles.

For each pair of dimension 1 in the persistence diagram, there is a 1-cycle representing the boundary of the corresponding homology class as appeared in the filtration. Persistence pairs and cycles can be accessed by reading the VTK files with `meshio`.

In [2]:
import meshio
ppairs = meshio.read("./data/vtk-original/0/3_pd.vtk")

The persistence diagram is stored as a collection of lines. 
Each line represents a persistence pair. The line end-points are geometrically located where the homology class was born/died.

For each line of index `i` we can access:
- the persistence value associated with the pair (`ppairs.cell_data["Persistence"][i]`)
- the dimension of the homology class (`ppairs.cell_data["Type"][i]`)
- the end-points of the line (`ppairs.cells[0].data[i][0]`,`ppairs.cells[0].data[i][1]`)


For each point of index `p` we can access:
- the dimension of the corresponding simplex (`ppairs.point_data["Type"][p]`)
- the filtration value (`ppairs.point_data["Filtration"][p]`)
- the point position in the original domain (`ppairs.points[p]`)

In [3]:
import meshio
cycle = meshio.read("./data/vtk-original/0/3_cycles.vtk")

Since we are analyzing 2D images, we only have 1-dimensional cycles to represent the homology class. Then, each cycle is stored as a collection of lines. 

For each line of index `i` we can access:
- the unique id of the cycle this edge belongs to (`cycle.cell_data["CycleId"][i]`). Moreover, the CycleID indicates the persistence pair corresponding to the cycle (`ppair.cell_data[cycle.cell_data["CycleId"][i]]`) 
- the persistence value associated to the persistence pair (`cycle.cell_data["Filtration][i]`)


In [4]:
import os
import meshio
import numpy as np
import pandas as pd
import gtda.plotting as gplotting
import gtda.diagrams as gdiagrams
import plotly.express as px
import plotly.graph_objs as gobj
from plotly.offline import init_notebook_mode, iplot
from skimage import io
from scipy import ndimage


DATA_DIR = os.path.join(os.getcwd(), "data")              # path of the data folder
IMG_DIR = os.path.join(DATA_DIR, "mnist_png")             # path of the image folder
VTK_DIR = os.path.join(DATA_DIR, "vtk-distancetransform") # path of the vtk files originated with the distance transform

init_notebook_mode(connected=True)

We have defined a couple of helper functions that can be used to extract information from these files:
- `getPersCycles()`: Extract information about the geometry of each cycle stored in a vtk file.
- `getPersDiagram()`: Create a persistence diagram starting from the persistence pairs stoored in a vtk file.

In [5]:
def getPersCycles(cycleFile):
    """
    Get persistence cycles from the vtk file storing the persistence cycles.
    
    Parameters
    ----------
    cycleFile: string
        The path of the vtk file storing the persistence cycles.
    
    Returns
    ---------
    ordered_cycles: dict
        The dictionary storing the cycle information.
    """
    im = meshio.read(cycleFile)
    
    # for each cycle id, save the list of edges
    ordered_cycles = {}
    array_of_cycles = {}
    for i in range(len(im.cell_data['CycleId'][0])):
        index = str(im.cell_data['CycleId'][0][i])

        if(index not in array_of_cycles):
            array_of_cycles[index] = []


        line = im.cells_dict['line'][i]
        array_of_cycles[index].append([line[0].tolist(), line[1].tolist()])
    
    # inner function to search for the next point
    def searchNext(all_lines, visited, point, prev):
        for i in range(len(all_lines)):
            if(not visited[i]):
                line = all_lines[i]
                if(point == line[0] and prev != line[1]):
                    visited[i] = True
                    return line[1]
                elif(point == line[1] and prev != line[0]):
                    visited[i] = True
                    return line[0]
        
    # for each cycle reorder the vertices so to have them ready to be plotted with THREE.js
    for k in array_of_cycles:
        val = array_of_cycles[k]
        visited = np.zeros(len(val))
        new_array = []

        first_point= val[0][0]
        prevP = val[0][0]
        nextP = val[0][1]

        visited[0] = True

        new_array.append(im.points[first_point].tolist())
        while(nextP != first_point):
            new_array.append(im.points[nextP].tolist())
            newP = searchNext(val, visited, nextP, prevP)
            prevP = nextP
            nextP = newP

        new_array.append(im.points[first_point].tolist())
        ordered_cycles[k] = new_array
        
    return ordered_cycles    

In [6]:
def getPersDiagram(diagramFile):
    """
    Get persistence diagram pairs from the vtk file storing persistence diagram.
    
    Parameters
    ----------
    diagramFile: string
        The path of the diagram vtk file.
        
    Returns 
    ----------
    pdiagram: array
        The numpy array storing the persistence diagram in the form (birth, death, dimension).
    """
    pdiagram = []
    try: 
        ppairs = meshio.read(diagramFile)
        k = 0
        for line in ppairs.cells[0].data: 
            v0 = line[0]
            v1 = line[1]
            f0 = ppairs.point_data["Filtration"][v0]    # birth
            f1 = ppairs.point_data["Filtration"][v1]    # death
            t = ppairs.cell_data["Type"][0][k]          # homology dimension
            k += 1
            # if t == 1: 
            pdiagram.append([f0, f1, t])
    except:
        print("No persistence pairs in the file")
    return np.asarray(pdiagram)

The plotting functions use the VTK files to produce visualization with Plotly.

The first step is that of visualizing components and cycles alone. To do so, we have implemented two functions to overlay such information to the image of the digit used to generate them. The function uses the `Scatter` class form Plotly.

The first function, `visualizeComponents`, simply reads the persistence pairs from the persistence diagram and plots them in the original image domain. For each point, it also indicates their lifespan

In [7]:
def visualizeComponents(img, diagramFile):
    """
        Plot persistence cycles on the original image file.

        Parameters
        ----------
        img: matrix
            The matrix storing the image. 
        diagramFile: string
            The path of the vtk file storing the persistence pairs.

        Returns
        ----------
        fig : :class:`plotly.graph_objects.Figure` object
            Figure showing where each component was born.
    """
    
    fig = px.imshow(img, color_continuous_scale='gray')

        
    ppairs = meshio.read(diagramFile)
    k = 0
    xcoords = []
    ycoords = []
    births = []
    deaths = []
    for line in ppairs.cells[0].data: 
        if(ppairs.cell_data["Type"][0][k] == 0):
            xcoords.append(ppairs.points[line[0]][1])
            ycoords.append(ppairs.points[line[0]][0])
            births.append(ppairs.point_data["Filtration"][line[0]])
            deaths.append(ppairs.point_data["Filtration"][line[1]])
    
    
    df = pd.DataFrame(data={"x":xcoords, "y":ycoords, "birth":births, "death":deaths})
    fig.add_trace(gobj.Scatter(x=df["x"], y=df["y"], text=df["death"]-df["birth"], hoverinfo="text", mode="markers"))
    
    head_tail = os.path.split(diagramFile)
    digit = head_tail[0][-1]
    filename = head_tail[1]
    fig.update_layout(height=400, width=400, title_text="Digit "+digit+" - Persistence Diagram file "+filename)
    fig.update_layout(coloraxis_showscale=False)
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
        
    
    return fig

With `visualizeComponets` we can plot components overlayed to the original image

In [8]:
digitPrefix = "0/10"
pngFile = os.path.join(IMG_DIR, digitPrefix+".png")
img = io.imread(pngFile)

diagramFile = os.path.join(VTK_DIR, digitPrefix+"_pd.vtk")

visualizeComponents(img,diagramFile)

or overlayed to filtration that originated the persistence pairs

In [9]:
digitPrefix = "0/10"
pngFile = os.path.join(IMG_DIR, digitPrefix+".png")
im = io.imread(pngFile)

inside = im > 0
outside = im <= 0

im1 = ndimage.morphology.distance_transform_edt(inside)
im2 = ndimage.morphology.distance_transform_edt(outside)

imf = im2-im1
imf = (imf - np.min(imf))/np.ptp(imf)

cycleFile = os.path.join(VTK_DIR, digitPrefix+"_pd.vtk")
visualizeComponents(imf, cycleFile)

The second function, `visualizeCycles`, uses a helper function `getPersCycles` to read the ordered sequence of vertices forming each cycle.

In [10]:
def visualizeCycles(img, cycleFile):
    """
    Plot persistence cycles on the original image file.
    
    Parameters
    ----------
    img: matrix
        The matrix storing the image. 
    cycleFile: string
        The path of the vtk file storing the persistence cycles.
    
    Returns
    ----------
    fig : :class:`plotly.graph_objects.Figure` object
        Figure showing the persistence cycles on top of the image file.
    """
    
    fig = px.imshow(img, color_continuous_scale='gray')
    
    try: 
        ordered_cycles = getPersCycles(cycleFile)
        coords = list(ordered_cycles.values())
        for cycle in coords:        
            flat_coords = np.asarray([item for item in cycle])
            fig.add_trace(gobj.Scatter(x=flat_coords[:,1], y=flat_coords[:,0], mode='lines'))
        
    except:
        print("No cycles to visualize!")
    
    # Adjust the properties of the figure
    head_tail = os.path.split(cycleFile)
    digit = head_tail[0][-1]
    filename = head_tail[1]
    
    fig.update_layout(height=400, width=400, title_text="Digit "+digit+" - Cycle file "+filename)
    fig.update_layout(coloraxis_showscale=False)
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    
    return fig


Also in this case, `visualizeCycles` accepts an image as input. This can be used to visualize cycles overlayed to the original image

In [11]:
digitPrefix = "8/110"
pngFile = os.path.join(IMG_DIR, digitPrefix+".png")
img = io.imread(pngFile)

cycleFile = os.path.join(VTK_DIR, digitPrefix+"_cycles.vtk")
visualizeCycles(img, cycleFile)

or overlayed to the filtration that originated the cycles

In [12]:
digitPrefix = "8/110"
pngFile = os.path.join(IMG_DIR, digitPrefix+".png")
im = io.imread(pngFile)

inside = im > 0
outside = im <= 0

im1 = ndimage.morphology.distance_transform_edt(inside)
im2 = ndimage.morphology.distance_transform_edt(outside)

imf = im2-im1
imf = (imf - np.min(imf))/np.ptp(imf)

cycleFile = os.path.join(VTK_DIR, digitPrefix+"_cycles.vtk")
visualizeCycles(imf, cycleFile)

# 3. Applications

Here we provide two examples where the direct visualization of cycles can be of help.

## 3.1. Definition of the most suitable filtration

The first scenario where the direct visualization of cycles could be useful is when we are deciding how to create a suitable filtration for the input data. In general, we would like to use the best filtration so to highlight/capture the main features of our input as persistence pairs. Different filtration will naturally lead to different features. The direct visualization of these features can help us in our choice.

For example, using the MNIST dataset, we may decide to filter each input image according to the original pixel values, or the opposite order, or to use some other transformation (e.g., distance transform).

![filtrations](pictures/filtrations.png)
Images generated with `./scripts/filtrations.py`

What filtration is the most suitable for our goal? We could better decide by looking at the features created.
In this example we pick one of the files encoding a digit `8`.

In [13]:
digitPrefix = "8/110"

Then select the original fitlration as defined by graylevel pixels values and we visualize the generated cycles.

In [14]:

VTK_DIR_ORIG = os.path.join(DATA_DIR, "vtk-original") 

pngFile = os.path.join(IMG_DIR, digitPrefix+".png")
img = io.imread(pngFile)
cycleFile = os.path.join(VTK_DIR_ORIG, digitPrefix+"_cycles.vtk")

visualizeCycles(img, cycleFile)

The same process is repeated negating the graylevel values of the input image

In [15]:
VTK_DIR_INV = os.path.join(DATA_DIR, "vtk-inverse")

pngFile = os.path.join(IMG_DIR, digitPrefix+".png")
img = io.imread(pngFile)
cycleFile = os.path.join(VTK_DIR_INV, digitPrefix+"_cycles.vtk")

visualizeCycles(img, cycleFile)

And finally using the distance transform.

In [16]:
pngFile = os.path.join(IMG_DIR, digitPrefix+".png")
img = io.imread(pngFile)
cycleFile = os.path.join(VTK_DIR, digitPrefix+"_cycles.vtk")

visualizeCycles(img, cycleFile)

As we can notice, the filtration based on the origina graylevel values is not necesseraly the one better highlighting the features of the input digit. In this case the filtration starts from the dark pixels thus creating a large cycle `trace 6` which is later broken in smaller parts as soon as the lighter pixels are introduced.

The filtration based on the inverse of the of the graylevel values does a better job at localyzing the two loops in the `8` digit but many other cycles are created due to noise in the input image.

The filtration based on the distance transform is similar to the previous but less noisy, and thus generates fewer cycles.


Regardless of our final choice, the direct visualization of the cycles can help us deciding which one is the best filtration for the features we would like to recognize in our input data.

## 3.2. Unsupervised analysis with topological features

As an example where visualizing cycles could be of help, we consider the unsupervised analysis of data by means of TDA descriptors. 

For each input image in the MNIST dataset we compute a descriptor by means of `PersistenceImages`. Persistence Images are computed using the function provided by [Giotto-TDA](https://giotto-ai.github.io/gtda-docs/0.2.1/modules/generated/diagrams/representations/gtda.diagrams.PersistenceImage.html). 


Here we experienced some issue with getting the correct persistence image when there is only 1 persistence pair in the diagram. Based on the filtration we used in the previous step, we adapt the `_bin()` function from `gtda.diagrams._utils` by manually setting the minimum and maximum value. Since the function is called in `fit()` function of `PersistenceImage` class, we also create a variation of `fit()` function.

In [17]:
from gtda.diagrams._utils import _subdiagrams, _sample_image, _homology_dimensions_to_sorted_ints

def bins(X, metric, n_bins=100, homology_dimensions=None, **kw_args):
    if homology_dimensions is None:
        homology_dimensions = sorted(np.unique(X[0, :, 2]))
    # For some vectorizations, we force the values to be the same + widest
    sub_diags = {dim: _subdiagrams(X, [dim], remove_dim=True)
                 for dim in homology_dimensions}
    # For persistence images, move into birth-persistence
    if metric == 'persistence_image':
        for dim in homology_dimensions:
            sub_diags[dim][:, :, [1]] = sub_diags[dim][:, :, [1]] \
                - sub_diags[dim][:, :, [0]]
    
    min_vals = {dim: np.asarray([0, 0])
                for dim in homology_dimensions}
#     max_vals = {dim: np.asarray([1, 1])
#                 for dim in homology_dimensions}
    max_vals = {dim: np.max(sub_diags[dim], axis=(0, 1))
                for dim in homology_dimensions}
    
    if metric in ['landscape', 'betti', 'heat', 'silhouette']:
        #  Taking the min(resp. max) of a tuple `m` amounts to extracting
        #  the birth (resp. death) value
        min_vals = {d: np.array(2*[np.min(m)]) for d, m in min_vals.items()}
        max_vals = {d: np.array(2*[np.max(m)]) for d, m in max_vals.items()}

    # Scales between axes should be kept the same, but not between dimension
    all_max_values = np.stack(list(max_vals.values()))
    if len(homology_dimensions) == 1:
        all_max_values = all_max_values.reshape(1, -1)
    global_max_val = np.max(all_max_values, axis=0)
    max_vals = {dim: np.array([max_vals[dim][k] if
                               (max_vals[dim][k] != min_vals[dim][k])
                               else global_max_val[k] for k in range(2)])
                for dim in homology_dimensions}
    
    samplings = {}
    step_sizes = {}
    for dim in homology_dimensions:
        samplings[dim], step_sizes[dim] = np.linspace(
            min_vals[dim], max_vals[dim], retstep=True, num=n_bins)
    if metric in ['landscape', 'betti', 'heat', 'silhouette']:
        for dim in homology_dimensions:
            samplings[dim] = samplings[dim][:, [0], None]
            step_sizes[dim] = step_sizes[dim][0]
    return samplings, step_sizes


def fit(pimgObj, X):
    if pimgObj.weight_function is None:
        pimgObj.effective_weight_function_ = np.ones_like
    else:
        pimgObj.effective_weight_function_ = pimgObj.weight_function

    # Find the unique homology dimensions in the 3D array X passed to `fit`
    # assuming that they can all be found in its zero-th entry
    homology_dimensions_fit = np.unique(X[0, :, 2])
    pimgObj.homology_dimensions_ = \
        _homology_dimensions_to_sorted_ints(homology_dimensions_fit)
    pimgObj._n_dimensions = len(pimgObj.homology_dimensions_)

    pimgObj._samplings, pimgObj._step_size = bins(
        X, "persistence_image",  n_bins=pimgObj.n_bins,
        homology_dimensions=pimgObj.homology_dimensions_
        )
    pimgObj.weights_ = {
        dim: pimgObj.effective_weight_function_(samplings_dim[:, 1])
        for dim, samplings_dim in pimgObj._samplings.items()
        }
    pimgObj.samplings_ = {dim: s.T for dim, s in pimgObj._samplings.items()}

We now use the new functions to compute the persistence images for the dataset.

In [18]:
def getPersImage(diagramFile, dimension=None, plot=False):
    """
    Get persistence image from the vtk file storing persistence diagram.
    
    Parameters
    ----------
    diagramFile: string
        The path of the diagram vtk file.
    dimensions: int
        The homology dimension.
    plot: bool
        If True, plot the computed persistence image.
        
    Returns 
    ----------
    persImage: array
        The numpy array storing the persistence image.
    """
    num_pixels = 10
    pdiagram = getPersDiagram(diagramFile)
    if pdiagram.size == 0:
        return np.zeros((num_pixels, num_pixels))
    pdiagram3D = pdiagram.reshape(1, -1, 3)
    
    pimgr = gdiagrams.PersistenceImage(sigma=0.1, n_bins=num_pixels)
    fit(pimgr, pdiagram3D)
    persImage = pimgr.transform(pdiagram3D)
    
    if dimension is None: 
        if plot:
            pimgr.plot(persImage)
        return persImage[0]
    
    if dimension >= persImage.shape[1]:
        return np.zeros((num_pixels, num_pixels))
    
    if plot:
        fig = pimgr.plot(persImage, homology_dimension_idx=dimension)
        fig.show()
    return persImage[0][dimension]
    

persImages = []
fileLabels = []
for i in range(10):
    pdiagDir = os.path.join(VTK_DIR, str(i))
    if os.path.exists(pdiagDir):
        files = [f for f in os.listdir(pdiagDir) if "_pd.vtk" in f]
        files.sort()
        for file in files:
            filePath = os.path.join(pdiagDir, file)
            persImages.append(getPersImage(filePath, 1))
            fileLabels.append(str(i) + '/' + file)


No persistence pairs in the file
No persistence pairs in the file


Each persistence image is then vectorized and used as a high-dimensional descriptor of the input datum. To explore similarity and differences among these descriptors we use t-SNE to generate a lower dimensional embedding for such descriptors

In [19]:
import scipy.spatial.distance as ssdist
from sklearn import manifold
from sklearn.metrics import pairwise_distances

def getEmbeddings(persImages):
    """
    Get the embeddings from the distance matrix using different methods.

    Parameters
    ----------
    persImage: array_like
        The array storing persistence images of same size.
    
    Returns
    ----------
    embeddings: ndarray 
        The array containing embeddings computed with t-SNE.
    """
    pers_images = np.asarray(persImages)
    if pers_images.ndim != 3:
        print("The dimension of the input array should be 3!\n")
        return -1
    
    pers_images = pers_images.reshape(len(pers_images), -1)
    tSNEMethod = manifold.TSNE(n_components=2, init="pca", random_state=0)
    embeddings = tSNEMethod.fit_transform(pers_images)
    
    return embeddings

digitLabels = [f[0] for f in fileLabels]
Y = getEmbeddings(persImages)
# plot the scattering 
df = pd.DataFrame(data={"x":Y[:,0], "y":Y[:,1], "digit": digitLabels, "file": fileLabels})
embeddingPlot = px.scatter(df, x="x", y="y", color="digit", hover_data=["x", "y", "file"])
iplot(embeddingPlot)

The lower dimensional embedding provides information regarding possible groups created by using the input descriptors. In this case, since we are using descriptors based on 1-cycles only, the main difference captured by t-SNE relates to the number of cycles found in each filtration.

At this point, natural questions may arise like: "why two points are close/far away in the 2D plot" or "how similar the original descriptors are?". Cycles can help answering these questions by providing an explicit representation of the homology classes that contributed to each persistence image.


### Cluster with 2 major cycles

![scaled plot 1](./pictures/plot_example_1.png)

The figure above shows the scaled view of the bottom right part of the embedding plot, where most digits are 8 with two cycles. To understand why some digits with 1 cycle appear in the cluster, we picked three example points from the figure. The first one is in purple which is associated with the file `3/18_pd.vtk`; the second one is in coral pink which is associated with the file `6/366_pd.vtk`; the last one is in pink which is associated with the file `8/401_pd.vtk`. We can then visualize the cycle information from the corresponding cycle vtk files. 

In [20]:
digitList = ["8/401", "3/18", "6/366"]

for digitPrefix in digitList:
    pngFile = os.path.join(IMG_DIR, digitPrefix+".png")
    img = io.imread(pngFile)

    cycleFile = os.path.join(VTK_DIR, digitPrefix+"_cycles.vtk")
    iplot(visualizeCycles(img, cycleFile))

From the images above, we can see that the digit 3 and 6 have two major cycles, which makes them accidently clustered with digit 8. We can find the similar cause for other points in the figure too.

### Cluster without any major cycles

![scaled plot 2](./pictures/plot_example_2.png)

The figure above shows the scaled view of the leftmost part of the embedding plot, where most digits are without any cycles. To understand why some digits with 1 cycle appear in the cluster, we picked three example points. The first one is in green which is associated with the file `2/208_pd.vtk`; the second one is in coral pink, which is associated with the file `6/559_pd.vtk`; the last one is in yellow which is associated with the file `9/20_pd.vtk`. We can then visualize the cycle information from the corresponding cycle vtk files. 

In [21]:
digitList = ["2/208", "6/559", "9/20"]

for digitPrefix in digitList:
    pngFile = os.path.join(IMG_DIR, digitPrefix+".png")
    img = io.imread(pngFile)

    cycleFile = os.path.join(VTK_DIR, digitPrefix+"_cycles.vtk")
    iplot(visualizeCycles(img, cycleFile))

No cycles to visualize!


From the images above, we can see that no major cycles are deteced in these digits, which makes them end up to be in the same cluster. We can find the similar cause for other points in the figure too.

# 3. Benchmark


Our apprach uses Discrete Morse Theory to encode the original chain complex by means of a Forma gradient. In practice, this allows for a more compact encoding of the information normally contained in the boundary matrix. Everything is implemented in plain C++ and integrated in the [TTK library]().

The steps for computing persistent homology with our modules are the following:

1. triangulate the input image (feature provided by [TTK](https://github.com/topology-tool-kit/ttk/tree/dev/core/vtk/ttkTriangulationReader))
2. compute the Forman gradient according to the algorithm by [Robins et al., 2011](https://ieeexplore.ieee.org/document/5766002) (implemented [here](https://github.com/IuricichF/PersistenceCycles/tree/main/ttk-0.9.7/core/base/formanGradient))
3. compute the chain complex represented by the Forman gradient (implemented [here](https://github.com/IuricichF/PersistenceCycles/blob/b68ae3ebc218ed69babeee5c1e4ac7f5a89564cd/ttk-0.9.7/core/base/fG_PersistentHomology/FG_PersistentHomology_template.h#L37))
4. use the standard algorithm to compute persitent homology on the chain complex (implemented [here](https://github.com/IuricichF/PersistenceCycles/tree/main/ttk-0.9.7/core/base/boundaryMatrix))
5. for each pair reconstruct the cycle (implemented [here](https://github.com/IuricichF/PersistenceCycles/blob/b68ae3ebc218ed69babeee5c1e4ac7f5a89564cd/ttk-0.9.7/core/base/fG_PersistentHomology/FG_PersistentHomology_template.h#L318))

The method turns out very efficient even if we are just interested in computing persistent homology. On the larger image we tested it (a 3D image of size [500x500x100]) the computation of persistent homology takes 66.5 seconds and 1.09GB of memory using 48 threads on a desktop computer with two 3.00GHz Intel Xeon 6136 CPUs and 64GB of memory. As a comparison, [DIPHA](https://github.com/DIPHA/dipha) requires 69.4 seconds and 29.7GB on the same machine.


Mapping information from the persistent diagram back to the original domain is a feature rarely offered by TDA packages. In part, this is due to the algorithms used to speedup persistent homology computation. Since they distrupt the information contained in the boundary matrix, it becomes impossible to reconstruct cycles.

Finding the location of the simplices responsible for a birth or death is simpler task. To the best of our knowledge only [TTK](https://topology-tool-kit.github.io/index.html) and [HomCloud](https://homcloud.dev/basic-usage.en.html) provide such feature.

Regarding the computation of cycles, [Eeirene](https://github.com/Eetion/Eirene.jl), a package developed in Julia, provides the computation of representative cycles when working on Vietoris-Rips filtrations.

# 4. Limitations and Perspectives

## Limitations of our method

To build up our notebook we had to compute persistence diagrams and corresponding cycles outside of Giotto-TDA. We also needed persistence pairs to be aligned with the cycles computed.

Our currenet implementation presents some limitation since it is designed to work with simplicial complexes up to dimension 3 only. This includes triangle meshes, tetrahedral meshes, or 2D and 3D images that are automatically triangulated by TTK.

## Limitations of Giotto-TDA

One problem we encountered wth Giotto-TDA was when computing persistence images on persistence diagrams formed by a single persistence pair. As the example shown below, we get an empty persistence image when there is only one persistence pair. 


In [22]:
exampleDiagram = np.array([[[0.1, 0.2, 1]]])
pimgr = gdiagrams.PersistenceImage(sigma=0.01, n_bins=10)
pimgr.fit(exampleDiagram)
persImage = pimgr.transform(exampleDiagram)
print(persImage)
iplot(pimgr.plot(persImage[0]))

[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]]


The root cause of the problem is in the `_bin()` function from `gtda.diagrams._utils`, and specifically the code to find the minimum and maximum persistence pair from the persistence diagram for generating the sampling slices and step sizes.

```python
def _bin(X, metric, n_bins=100, homology_dimensions=None, **kw_args):
    ......
    min_vals = {dim: np.min(sub_diags[dim], axis=(0, 1))
                for dim in homology_dimensions}
    max_vals = {dim: np.max(sub_diags[dim], axis=(0, 1))
                for dim in homology_dimensions}
    ......
    samplings = {}
    step_sizes = {}
    for dim in homology_dimensions:
        samplings[dim], step_sizes[dim] = np.linspace(
            min_vals[dim], max_vals[dim], retstep=True, num=n_bins
            )
        
    ......
```

One possible fix would be to add a parameter for `PersistenceImage` class which allows the range to be manually set, and then to be used in the `_bin()` function. As a comparison, we provide the persistence image generated with this approach.

In [23]:
exampleDiagram = np.array([[[0.1, 0.2, 1]]])
pimgr = gdiagrams.PersistenceImage(sigma=0.01, n_bins=10)
fit(pimgr, exampleDiagram)
persImage = pimgr.transform(exampleDiagram)
with np.printoptions(precision=2, suppress=True, linewidth=np.inf):
    print(persImage)
iplot(pimgr.plot(persImage))

[[[[   0.      0.      0.      0.      0.      0.08    6.15  134.74  858.49 1591.55]
   [   0.      0.      0.      0.      0.      0.04    3.32   72.68  463.08  858.49]
   [   0.      0.      0.      0.      0.      0.01    0.52   11.41   72.68  134.74]
   [   0.      0.      0.      0.      0.      0.      0.02    0.52    3.32    6.15]
   [   0.      0.      0.      0.      0.      0.      0.      0.01    0.04    0.08]
   [   0.      0.      0.      0.      0.      0.      0.      0.      0.      0.  ]
   [   0.      0.      0.      0.      0.      0.      0.      0.      0.      0.  ]
   [   0.      0.      0.      0.      0.      0.      0.      0.      0.      0.  ]
   [   0.      0.      0.      0.      0.      0.      0.      0.      0.      0.  ]
   [   0.      0.      0.      0.      0.      0.      0.      0.      0.      0.  ]]]]


## Perspectives


We think it would be very useful to integrate our module in Giotto-tda. While we have not taken steps in this directions yet, the integration should not be too challenging. 

The step requiring the most work should be adapting the input and output produced by the main functions. Regarding the input, we should update TTK Triangulation so to let it read and triangulate a numpy array. Regarding outputs, we can simply convert persistence diagrams in the format generated by Giotto-TDA and define a new object to store cycles.

Aside from adapting our implementation, we should also design visualization functions for 2-cycles in 3D. In practice, these are surfaces representing the boundary of the voids created by the filtration which should be fairly easy to achieve in Plotly.